# Basics on Python-Blosc2

Python-Blosc2 is a thin wrapper for the C-Blosc2 format and compresion library.  It allows to easily and quicly create, append, insert, update and delete data and metadata in a super-chunk container (SChunk class).

In [1]:
import blosc2
import numpy as np

## Create a new SChunk instance

Let's configure the parameters that are different from defaults:

In [2]:
cparams = {
    "compcode": blosc2.Codec.LZ4,
    "typesize": 4}
storage = {
    "contiguous": True,
    "urlpath": "myfile.b2frame",
    "cparams": cparams}

And let's remove a possible existing serialized super-chunk (frame):

In [3]:
blosc2.remove_urlpath("myfile.b2frame")

Now, we can already create a SChunk!

In [4]:
schunk = blosc2.SChunk(chunksize=40_000, **storage)
schunk

Great! So you have created your first super-chunk with your desired compression codec and typesize, and it is going to be persistent on disk.

## Append and read data

We are going to add some data.  That's is easy too:

In [5]:
for i in range(10):
    buffer = i * np.arange(10_000, dtype="int32")
    nchunks = schunk.append_data(buffer)
    assert nchunks == (i + 1)

In [6]:
!ls -lh myfile.b2frame

-rw-r--r--  1 faltet  staff    11K Jun 27 17:17 myfile.b2frame


So, while we have added 10 chunks of 40 KB each, the data size of the frame on-disk is just 11 KB.  This is how compression is helping you to use less resources.

Now, let's read the chunks from disk:

In [7]:
dest = np.empty(10_000, dtype="int32")
for i in range(10):
    chunk = schunk.decompress_chunk(i, dest)
    check = i * np.arange(10_000, dtype="int32")
    np.testing.assert_equal(dest, check)

So we have read all the data, and made sure that we read it correctly.

## Updating and inserting

First, let's update the first chunk:

In [8]:
data_up = np.arange(10_000, dtype='int32')
chunk = blosc2.compress2(data_up)
schunk.update_chunk(nchunk=0, chunk=chunk)

10

And then, insert another one at position 4:

In [9]:
schunk.insert_chunk(nchunk=4, chunk=chunk)

11

In this case the return value is the new number of chunks in the super-chunk.

Check that the data has been correctly inserted:

In [10]:
chunk = schunk.decompress_chunk(4, dest)
np.testing.assert_equal(dest, data_up)

## Add user meta info

The user can also add meta-information via the `vlmeta` accessor.  `vlmeta` stands for "variable length metadata", and, as the name suggests, it is meant to store general, variable length data (incidentally, this is more flexible than what you can store as regular data, which is always the same `typesize`).

`vlmeta` follows the dictionary interface, so adding info is as easy as:

In [11]:
schunk.vlmeta['info1'] = 'This is an example'
schunk.vlmeta['info2'] = 'of user meta handling'
schunk.vlmeta.getall()

{'info1': 'This is an example', 'info2': 'of user meta handling'}

You can also delete an entry as you would do with a dictionary:

In [12]:
del schunk.vlmeta['info1']
schunk.vlmeta.getall()

{'info2': 'of user meta handling'}

That's all for now.  There are more examples in the examples directory for you to explore.  Enjoy!